# Add Dependences

In [1]:
include("../../src/forces_func.jl")
include("../../src/struct_data.jl")
include("../../src/test.jl")
include("../../src/forces.jl")
include("../../src/neighbor.jl")
include("../../src/run_event.jl")

@make_struct_func Cubic

force_func (generic function with 1 method)

# Add Data

In [2]:
parameters = Model(
    Cubic(
        μ₁       = 0.1,
        rₘᵢₙ     = 2,
        rₘₐₓ     = 3
    ),
    Contractile(
        fₚ       = 0.0000001
    ),
    Time(
        t_f      = 100000, 
        dt       = 0.1
    ),
    Neighbor(
        n_knn    = 50,
        nn       = 12
    ),
    Geometry(
        R_agg    = 9,
        position = [
            0.0    0.0    0.0
        ]
    ),
    Simulation(
        n_text   = 200,
        path     = "Test-fusion.xyz"
    )
)

## Var before func
println("\n Loading Force Function")
CUDA.@time force(r) =  force_func(parameters.Force::Cubic, r)


 Loading Force Function
  0.000051 seconds (26 CPU allocations: 1.714 KiB)


force (generic function with 1 method)

## Starting Data

In [3]:
# Loading Data
CUDA.@time X = Float32.(readdlm("../../data/init/Sphere/$(Int(parameters.Geometry.R_agg)).xyz")[3:end,2:end]) |> cu
# Generating Struct
CUDA.@time Agg = Aggregate(Neighbor(50, 12), PositionCell(X));

  1.725348 seconds (2.28 M CPU allocations: 132.495 MiB, 3.40% gc time) (1 GPU allocation: 6.305 KiB, 3.03% memmgmt time)
  2.364483 seconds (2.30 M CPU allocations: 134.826 MiB, 2.33% gc time) (12 GPU allocations: 4.813 MiB, 0.05% memmgmt time)


## run_event -> one_aggregate

In [4]:
# Simulation One Aggregate
# CUDA.@time one_aggregate(true, parameters, Cubic(1.0, 2.0, 3.0), Agg)

## run_event -> fusion

In [5]:
# Simulating Two Aggregate
CUDA.@time fusion(true, parameters, Cubic(1.0, 2.0, 3.0), Agg)

Finding Equilibrium in one Aggregate


Progress:   1%|▍                        |  ETA: 2:16:30m02merror in running finalizer: InterruptException()
maybe_collect at /buildworker/worker/package_linux64/build/src/julia_threads.h:313 [inlined]
jl_gc_pool_alloc at /buildworker/worker/package_linux64/build/src/gc.c:1204
default_stream at /home/nicomosty/.julia/packages/CUDA/DfvRa/lib/cudadrv/stream.jl:32 [inlined]
unsafe_finalize! at /home/nicomosty/.julia/packages/CUDA/DfvRa/src/array.jl:99
_jl_invoke at /buildworker/worker/package_linux64/build/src/gf.c:2237 [inlined]
jl_apply_generic at /buildworker/worker/package_linux64/build/src/gf.c:2419
jl_apply at /buildworker/worker/package_linux64/build/src/julia.h:1703 [inlined]
run_finalizer at /buildworker/worker/package_linux64/build/src/gc.c:278
jl_gc_run_finalizers_in_list at /buildworker/worker/package_linux64/build/src/gc.c:365
run_finalizers at /buildworker/worker/package_linux64/build/src/gc.c:394
jl_gc_collect at /buildworker/worker/package_linux64/build/src/gc.c:3260
gc at 